In [1]:
## Imports
import urllib.request
import json
import matplotlib.pyplot as plt
import pickle
import re


In [2]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, 2)

def load_object(filename):
    output = 0
    with open(filename, 'rb') as fp:
        output = pickle.load(fp)
    return output


In [3]:
def read_json(html_extension,type_url):
    if ("http" not in html_extension):
        html = 'http://api.openparliament.ca'+ html_extension    # api.openparliament.ca is the root to all nested urls
    else:
        html = html_extension
    
    if type_url == "other":
        with urllib.request.urlopen(html) as url:
            data = json.loads(url.read().decode())
            content = data['objects']
            pagination = data['pagination']
            return pagination, content
        
    if type_url == "single":
        while (True):
            try:
                with urllib.request.urlopen(html) as url:
                    data = json.loads(url.read().decode())
                    return data
            except:
                print("url response not well, try again")
        


In [4]:
def nested_url(pagination,content):
    list_url = []
    while isinstance(pagination['next_url'],str):
        list_url.append(pagination['next_url'])
    
        pagination, content = read_json(pagination['next_url'],'other')
    return list_url

In [ ]:
first_mps = '/politicians/?include=all&format=json'
pagination,content = read_json(first_mps,'other')     #please treat tuples properly and i mean separate the elements
b=nested_url(pagination,content)
print (b)

In [5]:
def mp_info_all(pagination, content):
    df = {}
    politicians = content
    
    while isinstance(pagination['next_url'],str):
        
        if (not isinstance(politicians, list)):
            break
       
        for i in range(0,len(politicians)):
            poli_url = politicians[i]['url']                               # mp url from list of mps
            name = poli_url.split("/politicians/",1)[1][:-1]        # split of string and removal of /
            info_mp = read_json(poli_url+'?format=json','single')
            list_s_date = []
            list_e_date = []
            list_prov = []
            list_party = []
            memberships = info_mp['memberships']

            for j in range(0, len(memberships)):
                if memberships[j]['end_date'] is None:
                    list_e_date.append('still active')
                else:
                    list_e_date.append(memberships[j]['end_date'])

                list_s_date.append(memberships[j]['start_date'])
                list_prov.append(memberships[j]['riding']['province'])
                list_party.append(memberships[j]['party']['short_name']['en'])
            df.update({name: {'province':list_prov,'party':list_party,'start date':list_s_date,'end date': list_e_date}})
            print ("number of MP: " + str(len(df)), end="\r")
        if isinstance(pagination['next_url'], str):
            pagination, content = read_json(pagination['next_url'],'other')
        politicians = content
        
            
            
    return df

In [ ]:
# first_mps = '/politicians/?include=all&format=json'
# pagination,content = read_json(first_mps,'other')

#{name: {'province':list_prov,'party':list_party,'start date':list_s_date,'end date': list_e_date}}
mps = mp_info_all(pagination, content)
save_object(mps, "mp_dict.pkl")
print (len(mps))

In [6]:
mps = load_object("mp_dict.pkl")
print (len(mps))


1014


In [ ]:
# http://api.openparliament.ca/bills/?sponsor_politician=ruth-ellen-brosseau&format=json

In [7]:

#input is a list of mp names only

#return a dictionary of {vote_url : sponsor}
def sponsor(mps):
    web = "http://api.openparliament.ca/bills/?sponsor_politician="
    j_format = "&limit=500&format=json"                               # set limit to 500 to avoir nested future url
    
    
    # {vote : sponsors}
    vote_sponsors = {}
    
    
    for i in range(0,len(mps)):
        v_url_total = []
        
        sponsor_url = web + str(mps[i]) + j_format
        pagination,content = read_json(sponsor_url,'other')
        
        bills = content
        More = True
                
        while (More):        
            for j in range(0,len(bills)):                                 # access to list of bills sponsored
                bill_id = bills[j]['url']
                bill_url = "http://api.openparliament.ca"+ bill_id + "?format=json"
                bill_content = read_json(bill_url, 'single')
                vote_urls = bill_content["vote_urls"]
                if (vote_urls is None):
                    continue
                v_url_total.extend(vote_urls)
            if isinstance(pagination['next_url'], str):
                pagination, content = read_json(pagination['next_url'],'other')   #everybody loves their pagination :)
            bills = content
            More = isinstance(pagination['next_url'],str)
        
        # vote_url: sponsor
        for vote_url in v_url_total:
            vote_sponsors[vote_url] = mps[i]
        
        print ("number of mps: " + str(i), end="\r")
            
    return vote_sponsors
            

In [8]:
vote_sponsors = sponsor(list(mps.keys()))
save_object(vote_sponsors, "vote_sponsors.pkl")
print (len(vote_sponsors))

1590er of mps: 1013


In [9]:
vote_sponsors = load_object("vote_sponsors.pkl")
# print (vote_sponsors)


In [10]:
'''
grab all the votes not by mp but by bill
'''
#http://api.openparliament.ca/votes/ballots/?vote=%2Fvotes%2F42-1%2F1344%2F&ballot=Yes&limit=500&format=json
def pos_votes(vote_sponsors, vote_type="Yes"):
    
    #year : edgelist
    temp_edgelist = {}
    idx = 0
    
    for vote_url in vote_sponsors:
        vote_urlname = re.split(' |/|\\\\',vote_url)
        session, vote = vote_urlname[2], vote_urlname[3]
        
        
        #http://api.openparliament.ca/votes/43-1/8/?format=json
        vote_html = vote_url+"?format=json"
        vote_content = read_json(vote_html, "single")
        date = vote_content["date"][0:4]
        
        ballot_url = '/votes/ballots/?vote=%2Fvotes%2F'+ session + '%2F' + vote + '%2F&limit=500&ballot=Yes&format=json'
        pagination, ballots = read_json(ballot_url, "other")
        
        for ballot in ballots:
            politician_url = ballot["politician_url"]
            politician = politician_url.split("/")[-2]
            if (date not in temp_edgelist):
                temp_edgelist[date] = [(politician,vote_sponsors[vote_url])]
            else:
                temp_edgelist[date].append((politician,vote_sponsors[vote_url]))
        
        idx = idx + 1
        
        print ("number of vote_url: " + str(idx), end="\r")
        
    return temp_edgelist
    
    
        
            
            
            
            
        
        
        
    
    
    
    
    


In [11]:
temp_edgelist = pos_votes(vote_sponsors, vote_type="Yes")
save_object(temp_edgelist, "temp_edgelist.pkl")
print (len(temp_edgelist))


14mber of vote_url: 1590


In [ ]:
'''
take as input a list of mp names
'''
#vote typpe: "Yes", "No", "Didn't vote", "Paired"
def all_votes(mps, vote_type="Yes", prev=None, num=-1):
    
    
    #date: {bill:[mp list]}
    if (prev is not None):
        vote_dict = prev
    else:
        vote_dict = {}
    
    
    
    web = "http://api.openparliament.ca/votes/ballots/?ballot=" + vote_type + "&politician=%2Fpoliticians%2F"
    json_format = "%2F&limit=1000&format=json"
    for i in range(num+1,len(mps)):
        pagination,content = read_json(web+mps[i]+json_format, "other")
        votes = content
        More= True
        
        #date:[bill_ids]
        date_dict = {}
        
        while (More):
            for j in range(0,len(votes)):                                 # votes of the specified type 
                vote_url = votes[j]["vote_url"]
                prefix = "http://api.openparliament.ca/"
                suffix = "?format=json"
                
                vote_info = read_json(prefix + vote_url + suffix, "single")
                bill_url = vote_info["bill_url"]
                if (bill_url is None):
                    continue
                date = vote_info["date"]
                bill_id = bill_url
                if (date not in date_dict):
                    date_dict[date] = [bill_id]
                else:
                    date_dict[date].append(bill_id)
                
            #check if there is more
            if isinstance(pagination['next_url'], str):
                    pagination, content = read_json(pagination['next_url'],'other')   #everybody loves their pagination :)
            votes = content
            More = isinstance(pagination['next_url'],str)
        
        for date in date_dict:
            for bill_id in date_dict[date]:
                if (date not in vote_dict):
                    vote_dict[date] = {bill_id:[mps[i]]}
                else:
                    if (bill_id not in vote_dict[date]):
                        vote_dict[date][bill_id] = [mps[i]]
                    else:
                        vote_dict[date][bill_id].append(mps[i])
        
        
        print ("number of mps: " + str(i), end="\r")
        save_object(vote_dict, "yes_votes.pkl")
    return vote_dict

In [ ]:
yes_votes = load_object("yes_votes.pkl")
print (len(yes_votes))
yes_votes = all_votes(list(mps.keys()))
print (len(yes_votes))
save_object(yes_votes, "yes_votes.pkl")


In [ ]:
url_all_politicians = "http://api.openparliament.ca/politicians/?include=current&format=json"
b = "http://api.openparliament.ca/votes/ballots/?ballot=Yes&politician=%2Fpoliticians%2Fziad-aboultaif%2F&limit=1000&format=json"

In [ ]:
def yes_votes(list_poli):
    yes_bills = []
    web = "http://api.openparliament.ca/votes/?limit=500&format=json"
    votes = read_json(web,'other')
    for i in range(0,len())
    web = "http://api.openparliament.ca/votes/ballots/?ballot=Yes&politician=%2Fpoliticians%2F"
    json_format = "%2F&limit=1000&format=json"
    for i in range(0,len(list_poli)):
        votes = read_json(web+list_poli[i]+json_format)
        for j in votes

In [ ]:
mps = mp_info(read_json(url_all_politicians,'other'))

In [ ]:
print(mps[1]['name'])

In [ ]:
print(len(mps))

In [ ]:
sponsorship = sponsor(mps)

In [ ]:
## RUN CELL TOMORROW MORNING
print(sponsorship[44])

In [ ]:
c = []
for x in sponsorship:
    bb = sponsorship[x]['name']
    c.append(bb)

In [ ]:
print(c)

In [ ]:
ben = read_json('http://api.openparliament.ca/politicians/4830/?format=json','poli')

In [ ]:
for i in range(0,len(ben['memberships'])):
    print(ben['memberships'][i]['start_date'])

In [ ]:
def bill_extraction(url_bills):
    next_bills = url_bills['pagination']['next_url']
    while next_bills != null:
        